In [1]:
!pip install tensorflow


  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl.metadata (807 bytes)
Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx 1.14.0 requires protobuf>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.15.0 requires tensorflow-estimator<2.16,>=2.15.0, but you have tensorflow-estimator 2.10.0 which is incompatible.


In [2]:
import os
import fitz
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:

# Fonction pour extraire le texte d'un PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text


# Chargement du CSV
def load_csv(file_path):
    df = pd.read_csv(file_path, sep=";", encoding="latin-1" , engine='python')
    if len(df.columns) == 1:
        df = pd.read_csv(file_path, sep=';')
    return df



In [4]:
# Chargement et préparation des données

csv_data = load_csv("./Data_train/csv/Tarifs_BForBank2024_sommaire.csv")
# print(csv_data)


text_pdf = extract_text_from_pdf("./Data_train/pdf/Tarifs BForBank2024 V8_WEB.pdf")
#print(text_pdf)


# Préparation des sommaires
summaries = []
for _, row in csv_data.iterrows():
    title = row.iloc[0]  # Titre principal (première cellule de chaque ligne)
    subtitles = [subtitle for subtitle in row.iloc[1:] if isinstance(subtitle, str) and pd.notna(subtitle)]
    summary = f"{title}\n" + "\n".join(subtitles)
    summaries.append(summary)

print("summaries", summaries)

In [5]:

# Tokenisation
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text_pdf] + summaries)

X = tokenizer.texts_to_sequences([text_pdf] * len(summaries))
y = tokenizer.texts_to_sequences(summaries)

# Padding
max_len = max(len(seq) for seq in X + y)
X = pad_sequences(X, maxlen=max_len, padding='post')
y = pad_sequences(y, maxlen=max_len, padding='post')

# Conversion de y en format one-hot
vocab_size = len(tokenizer.word_index) + 1
y = np.array([np.eye(vocab_size)[sequence] for sequence in y])




In [6]:
# Création du modèle LSTM
embedding_dim = 100
lstm_units = 128
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(X, y, epochs=10, batch_size=1, verbose=1)


Epoch 1/10
11/11 [==============================] - 9s 428ms/step - loss: 6.3100 - accuracy: 0.9077
Epoch 2/10
11/11 [==============================] - 5s 430ms/step - loss: 2.0878 - accuracy: 0.9990
Epoch 3/10
11/11 [==============================] - 5s 423ms/step - loss: 0.1944 - accuracy: 0.9990
Epoch 4/10
11/11 [==============================] - 5s 431ms/step - loss: 0.0374 - accuracy: 0.9990
Epoch 5/10
11/11 [==============================] - 5s 422ms/step - loss: 0.0221 - accuracy: 0.9990
Epoch 6/10
11/11 [==============================] - 5s 425ms/step - loss: 0.0184 - accuracy: 0.9990
Epoch 7/10
11/11 [==============================] - 5s 431ms/step - loss: 0.0170 - accuracy: 0.9990
Epoch 8/10
11/11 [==============================] - 5s 430ms/step - loss: 0.0162 - accuracy: 0.9990
Epoch 9/10
11/11 [==============================] - 5s 423ms/step - loss: 0.0156 - accuracy: 0.9990
Epoch 10/10
11/11 [==============================] - 5s 429ms/step - loss: 0.0151 - accuracy: 0.9990

In [20]:
def generate_summary(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded_seq)
    
    pred_text = []
    for timestep in pred[0]:
        word_index = np.argmax(timestep)
        word = tokenizer.index_word.get(word_index, "")
        if word:
            pred_text.append(word)
    
    # Formatage du sommaire prédit
    formatted_summary = " ".join(pred_text).strip()
    return formatted_summary

# Génération d'un sommaire pour le PDF d'origine
generated_summary = generate_summary(text_pdf)
print("Sommaire généré :")
print(generated_summary)

1/1 [==============================] - 0s 480ms/step
Sommaire généré :



In [19]:
print("\nTexte original :",  text_pdf)


Texte original : Tarifs applicables au 10 janvier 2024 
(Tarifs TTC)
Conditions tarifaires BforBank
Tarifs TTC - Au 10 janvier 2024 
2
Conditions tarifaires BforBank
* Les établissements de crédit ont la possibilité de présenter en tête de leurs plaquettes tarifaires un extrait standard de tarifs.
Extrait standard des tarifs*
Les dénominations des produits et services de l’Extrait standard des tarifs sont définies dans le «Glossaire des produits et services 
bancaires» en annexe des présentes.
Les tarifs ci-dessous sont hors offre groupée de services (package) et hors promotion ou tarif spécifique à une partie de la clientèle.
Liste des services
Prix en euro
Abonnement à des services de banque à distance (internet, téléphone fixe, SmS, etc.) 
hors coût du fournisseur d’accès internet
Gratuit
Abonnement à un produit offrant des alertes sur la situation du compte par email
Gratuit
Tenue de compte
Gratuit
Fourniture d’une carte de débit (carte de paiement à autorisation systématique) Vis